In [7]:
import cv2
import numpy as np

# 加载图像
img = cv2.imread('./mao/img.jpeg')

# 如果图像为空，退出
if img is None:
    print("无法加载图像")
    exit()


height, width = img.shape[:2]

gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 设置阈值，进行二值化，保留黑色部分
_, binary_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY_INV)

treeMask = binary_image == 255
# hole = binary_image == 255

# removed = img.copy()
tree = img.copy()

# 将原始图像中的白色部分去除
# removed[mask] = (255, 255, 255)  # 将保留部分改为白色
# tree[treeMask] = (255, 255, 255)  # 将保留部分改为白色


transparent_background = np.zeros((height, width, 4), dtype=np.uint8)
transparent_background[:, :, :3] = tree  # 前景 (BGR 通道)
transparent_background[:, :, 3] = binary_image     # Alpha 通道 (255 表示不透明，0 表示透明)
# 保存结果为 PNG 格式，其中透明通道保留
# cv2.imwrite('binary.png', binary_image)
cv2.imwrite('transparent_background.png', transparent_background)
# cv2.imwrite('tree.png', tree)


True

修复背景

In [28]:

repaired_image = cv2.inpaint(tree, binary_image, 20, cv2.INPAINT_TELEA)
cv2.imwrite('background.png', repaired_image)


True

根据data里的position来计算每个node的bounding box，之后将二值化的图像根据bbox切割，得到每个node的mask

In [9]:
import os, json

with open("./mao/data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def traverse_and_store(data, result, path=""):
    if not data:
        return

    # 将当前节点的 name 和 position 存储到结果列表
    result.append({"name": data["name"], "position": data.get("position")})

    # 遍历子节点
    for child in data.get("children", []):
        traverse_and_store(child, result, path)

results = []
traverse_and_store(data, results)

results

[{'name': '毛公', 'position': [[456, 76.562], [60, 130]]},
 {'name': '馬副', 'position': [[132, 418.56], [60, 125]]},
 {'name': '鄭玄', 'position': [[135, 710.5], [60, 116]]},
 {'name': '鄭衆', 'position': [[243, 426.56], [60, 120]]},
 {'name': '謝曼卿', 'position': [[350, 425.5], [60, 185]]},
 {'name': '衛宏', 'position': [[240, 1010.5], [64, 120]]},
 {'name': '徐巡', 'position': [[235, 1298.5], [68, 123]]},
 {'name': '賈𡽪', 'position': [[458, 1005.56], [60, 140]]},
 {'name': '逵', 'position': [[453, 1298], [60, 70]]},
 {'name': '孔子建', 'position': [[453, 437.56], [60, 187]]},
 {'name': '僖', 'position': [[453, 698.562], [60, 67]]},
 {'name': '孫期', 'position': [[567, 417.562], [60, 135]]},
 {'name': '尹敬', 'position': [[670, 423.562], [60, 125]]},
 {'name': '貫長卿', 'position': [[786, 680.562], [60, 176]]},
 {'name': '解迎年', 'position': [[790, 925.562], [60, 150]]},
 {'name': '徐敖', 'position': [[789, 1128.56], [60, 120]]},
 {'name': '陳俠', 'position': [[787, 1303.56], [60, 122]]}]

In [20]:
import os
import cv2
import json

image_file = "./mao/mask.png"  # 指定图片路径
output_dir = "./mao/masks"

image = cv2.imread(image_file, cv2.IMREAD_UNCHANGED)


for item in results:
    name = item["name"]
    bbox = item['position']
    x, y = int(bbox[0][0]), int(bbox[0][1])
    w, h = int(bbox[1][0]), int(bbox[1][1])
    x2, y2 = x + w, y + h

    # 裁切图片
    cropped = image[y:y2, x:x2]

    # 构建输出文件名
    output_path = os.path.join(output_dir, f"{name}.png")

    # 保存裁切的图片
    cv2.imwrite(output_path, cropped)
